In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import librosa

Data processing parameters 

In [2]:
num_aud = 62
audio_data = []
x = []
f_size = 512            # Frame size
h_size = int(f_size/2)  # Hop size
frame_num_gl = 170
data = np.zeros((100, h_size, frame_num_gl))

In [3]:
audio_data.append("wav\\129_a.wav")
# audio_data.append("D:\Voices\ALS\\008_a.wav")
# audio_data.append("D:\Voices\ALS\\008_i.wav")
# audio_data.append("D:\Voices\ALS\\020_a.wav")
# audio_data.append("D:\Voices\ALS\\020_i.wav")
# audio_data.append("D:\Voices\ALS\\021_a.wav")
# audio_data.append("D:\Voices\ALS\\021_i.wav")
# audio_data.append("D:\Voices\ALS\\022_a.wav")
# audio_data.append("D:\Voices\ALS\\022_i.wav")
# audio_data.append("D:\Voices\ALS\\024_a.wav")
# audio_data.append("D:\Voices\ALS\\024_i.wav")
# audio_data.append("D:\Voices\ALS\\025_a.wav")
# audio_data.append("D:\Voices\ALS\\025_i.wav")
# audio_data.append("D:\Voices\ALS\\027_a.wav")
# audio_data.append("D:\Voices\ALS\\027_i.wav")
# audio_data.append("D:\Voices\ALS\\028_a.wav")
# audio_data.append("D:\Voices\ALS\\028_i.wav")
# audio_data.append("D:\Voices\ALS\\031_a.wav")
# audio_data.append("D:\Voices\ALS\\031_i.wav")
# audio_data.append("D:\Voices\ALS\\032_a.wav")
# audio_data.append("D:\Voices\ALS\\032_i.wav")
# audio_data.append("D:\Voices\ALS\\039_a.wav")
# audio_data.append("D:\Voices\ALS\\039_i.wav")
# audio_data.append("D:\Voices\ALS\\042_a.wav")
# audio_data.append("D:\Voices\ALS\\042_i.wav")
# audio_data.append("D:\Voices\ALS\\046_a.wav")
# audio_data.append("D:\Voices\ALS\\046_i.wav")
# audio_data.append("D:\Voices\ALS\\048_a.wav")
# audio_data.append("D:\Voices\ALS\\048_i.wav")
# audio_data.append("D:\Voices\ALS\\052_a.wav")
# audio_data.append("D:\Voices\ALS\\052_i.wav")
# audio_data.append("D:\Voices\ALS\\055_a.wav")
# audio_data.append("D:\Voices\ALS\\055_i.wav")
# audio_data.append("D:\Voices\ALS\\058_a.wav")
# audio_data.append("D:\Voices\ALS\\058_i.wav")
# audio_data.append("D:\Voices\ALS\\062_a.wav")
# audio_data.append("D:\Voices\ALS\\062_i.wav")
# audio_data.append("D:\Voices\ALS\\064_a.wav")
# audio_data.append("D:\Voices\ALS\\064_i.wav")
# audio_data.append("D:\Voices\ALS\\068_a.wav")
# audio_data.append("D:\Voices\ALS\\068_i.wav")
# audio_data.append("D:\Voices\ALS\\072_a.wav")
# audio_data.append("D:\Voices\ALS\\072_i.wav")
# audio_data.append("D:\Voices\ALS\\076_a.wav")
# audio_data.append("D:\Voices\ALS\\076_i.wav")
# audio_data.append("D:\Voices\ALS\\078_a.wav")
# audio_data.append("D:\Voices\ALS\\078_i.wav")
# audio_data.append("D:\Voices\ALS\\080_a.wav")
# audio_data.append("D:\Voices\ALS\\080_i.wav")
# audio_data.append("D:\Voices\ALS\\084_a.wav")
# audio_data.append("D:\Voices\ALS\\084_i.wav")
# audio_data.append("D:\Voices\ALS\\092_a.wav")
# audio_data.append("D:\Voices\ALS\\092_i.wav")
# audio_data.append("D:\Voices\ALS\\094_a.wav")
# audio_data.append("D:\Voices\ALS\\094_i.wav")
# audio_data.append("D:\Voices\ALS\\096_a.wav")
# audio_data.append("D:\Voices\ALS\\096_i.wav")
# audio_data.append("D:\Voices\ALS\\098_a.wav")
# audio_data.append("D:\Voices\ALS\\098_i.wav")
# audio_data.append("D:\Voices\ALS\\100_a.wav")
# audio_data.append("D:\Voices\ALS\\100_i.wav")
# audio_data.append("D:\Voices\ALS\\102_a.wav")
# audio_data.append("D:\Voices\ALS\\102_i.wav")


# for i in range (len(audio_data)):
#     x_r, sr = librosa.load(audio_data[i], sr=None, mono=True, offset=0.0, duration=None)
#     x.append(x_r)

Analysing cycle

In [1]:
# freq = np.linspace(0, sr/2, h_size)

for i in range(1):  # len(audio_data)
    x_r, sr = librosa.load(audio_data[i], sr=None, mono=True, offset=0.0, duration=None)
    Ln = x[i].shape[0]
    frame_num = floor((Ln-f_size)/h_size + 1)

frame_num_gl = 1;

#for i in range(frame_num_gl):
    
    
#    frame_num_gl = frame_num_gl + frame_num;
    wnd = np.hamming(f_size)

    spec = np.zeros((h_size,frame_num))

    for N in range(frame_num):
        x_frame = x[h_size*N: h_size*N + f_size]
        x_wnd = x_frame*wnd

        X = np.fft.fft(x_wnd)
        X_abs = np.abs(X[0:h_size])

        spec[:,N] = np.log(X_abs)

   # plt.subplots(dpi=150)
   # plt.pcolor(spec[0:100,:],shading='flat')  

    
    #Запись фрагметов спектра в трехмерный массив 
    #Первая координата - номер фрейма
    N_i = math.ceil(N / 148);

    k = 0;
    
    for i in range(i + N_i):
        for j in range (148):
            if (j*(i+1) > N):
                   break
            data[[i], [k], [j]] = spec[[k],[j*(i+1)]];
            if(k < 100):
                k = k + 1 
            else :
                k = 100;
            k = 0;   
    print(i);
    
    

IndentationError: unexpected indent (2928091104.py, line 6)